<a href="https://colab.research.google.com/github/AltynayA/comp-math/blob/main/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Altynay Ayazbayeva | SE-2421
## Assignment 2 | Computational Mathematics

### Implementing

In [ ]:
import numpy as np
A = np.array( [
    [4,-1,-1,0],
    [-1,0,4,-1],
    [0,-1,-1,4],
    [-1,4,0,-1]])
b = np.array([-1, 7, 11, 3], dtype=float)

In [ ]:
np.linalg.solve(A, b)

array([1., 2., 3., 4.])

### Cramer's



In [ ]:
def cramers(A,b):
  sols = []
  A = np.array(A, float)
  b = np.array(b,float)
  det_A = np.linalg.det(A)
  if det_A == 0:
    return 0
    # number of columns
  n = len(b)

  for i in range(n):
    A_i = A.copy()
    A_i[:,i] = b
    x_i = np.linalg.det(A_i)/ det_A
    sols.append(x_i)
    # return f"x{i+1} = {x_i:.3f}"
  return sols

In [ ]:
solution = cramers(A, b)
# print(solution)
for i, val in enumerate(solution):
    print(f"x{i+1} = {val:.10g}")

x1 = 1
x2 = 2
x3 = 3
x4 = 4


### Gaussian

In [ ]:
def gaussian(A,b):
  n = len(b)
# copying into new matrix
  A = A.astype(float)
          # checking the pivot
  for i in range(n):
    if A[i, i] == 0:
      return "Zero pivot encountered"
    # Forward Elimination
    for j in range(i + 1, n):
      factor = A[j, i] / A[i, i]
      A[j, i:] -= factor * A[i, i:]
      b[j] -= factor * b[i]
# sub back
  x = np.zeros(n)
  for i in range(n - 1, -1, -1):
    x[i] = (b[i] - np.dot(A[i, i + 1:], x[i + 1:])) / A[i, i]
  return x

In [ ]:
print(gaussian(A,b))

[1. 2. 3. 4.]


### Gauss Jordan

In [ ]:
def gauss_jordan_elimination(A, b):
    # copying
    A = A.astype(float).copy()
    b = b.astype(float).copy()
    n = len(A)

    for i in range(n):
        # find pivot
        max_idx = np.argmax(np.abs(A[i:, i])) + i

        # swapping
        if max_idx != i:
            A[[i, max_idx]] = A[[max_idx, i]]
            b[[i, max_idx]] = b[[max_idx, i]]

        # normalizing (dividing)
        pivot = A[i, i]
        A[i] /= pivot
        b[i] /= pivot

        # removing other rows
        for j in range(n):
            if j != i:
                factor = A[j, i]
                A[j] -= factor * A[i]
                b[j] -= factor * b[i]

    return b  # b now stores the solution

In [ ]:
sol = gauss_jordan_elimination(A, b)
print(sol)

[ 3.60416667 12.86458333  2.55208333 -0.14583333]


### Jacobi’s iteration

 infinity norm of difference - np.inf

In [ ]:
import pandas as pd
def jacobiG(A,b,x0,max_iter,tol):
  A = np.asarray(A, dtype=float)
  b = np.asarray(b, dtype=float)
  x = np.asarray(x0, dtype=float).copy()
  n = len(b)

  for i in range(n):
    if A[i, i] == 0:
      return "Division by zero (diagonal)"

  iterations =[]

  for i in range(1,max_iter + 1):
    x_new = np.zeros(n)
# finding the sum
    for j in range(n):
      summ = 0
      for k in range(n):
        if k != j:
          summ += A[j,k] * x[k]
          # finding x_i

      x_new[j] = (b[j] - summ) / A[j, j]

    error = np.linalg.norm(x_new - x,np.inf)
    iterations.append((i, *x_new, error))

    if error < tol:
      print("tolerance reached")
      df = pd.DataFrame(iterations, columns=["Iteration"] + [f"x{i+1}" for i in range(n)] + ["Error"])
      return x_new,df

    x = x_new.copy()
  print(f"max iters {max_iter} reached")
  df = pd.DataFrame(iterations, columns=["Iteration"] + [f"x{i+1}" for i in range(n)] + ["Error"])
  return x,df


In [ ]:
initial_guess = np.array([0,0,0,0])
print(jacobi(A, b, x0, 100, 1e-8))

Division by zero (diagonal)


### Gauss-Seidel

better version of Jacobi, using the last recently computed values (x1,x2,x3)

In [ ]:
import pandas as pd

def gauss_seidel(A,b,x0,max_iter,tol):
  A = np.asarray(A, dtype=float)
  b = np.asarray(b, dtype=float)
  x = np.asarray(x0, dtype=float).copy()
  n = len(b)

  for i in range(n):
    if A[i, i] == 0:
      return "Zero diagonal (division)"

  iterations =[]

  for i in range(1, max_iter + 1):
     # storing old values
    x_old = x.copy()
    for j in range(n):
      summ = 0
      for k in range(n):
        if k < j:
          summ += A[j, k] * x[k]
        elif k > j:
          summ += A[j, k] * x_old[k]
          # finding x_i
      x[j] = (b[j] - summ) / A[j, j]

    error = np.linalg.norm(x - x_old,np.inf)
    iterations.append((i, *x, error))

    if error < tol:
      print("tolerance reached")
      df = pd.DataFrame(iterations,columns=["Iteration"] + [f"x{i+1}" for i in range(n)] + ["Error"]
            )
      return x,df

  print(f"max iters {max_iter} reached")
  # df = pd.DataFrame( iterations, columns=["Iteration"] + [f"x{i+1}" for i in range(n)] + ["Error"])
  df = pd.DataFrame(iterations, columns=["Iteration"] + [f"x{i+1}" for i in range(len(x))] + ["Error"])

  return x, df

In [ ]:
initial_guess = np.array([0,0,0,0])
gauss_seidel(A, b, initial_guess, 100, 1e-8)

'Zero diagonal (division)'

### Relaxation

faster convergence than Jacobi & Gauss-Seidel with relaxation paramtere (w)
w > 1

In [ ]:
def relaxation(A,b,x0,max_iter,tol, w):
  A = np.asarray(A, dtype=float)
  b = np.asarray(b, dtype=float)
  x = np.asarray(x0, dtype=float).copy()
  n = len(b)
  for i in range(n):
    if A[i, i] == 0:
      return "Zero diagonal (division)"
  iterations = []
  for i in range(1,max_iter + 1):
    # storing old values
    x_old = x.copy()
# updating each x
    for j in range(n):
      summ = 0
      for k in range(n):
        if k < j:
          #  updated values
          summ += A[j, k] * x[k]
        elif k > j:
          # pervious values
          summ += A[j, k] * x_old[k]
      x[j] = (1 - w) * x[j] + (w / A[j, j]) * (b[j] - summ)
    error = np.linalg.norm(x - x_old,np.inf)
    iterations.append((i, *x, error))

    if error < tol:
      print("tolerance reached")
      df = pd.DataFrame(iterations, columns=["Iteration"] + [f"x{i+1}" for i in range(n)] + ["Error"])
      return x, df
  print(f"max iters {max_iter} reached")
  df = pd.DataFrame(iterations, columns=["Iteration"] + [f"x{i+1}" for i in range(n)] + ["Error"])
  return x, df

In [ ]:
initial_guess = np.array([0,0,0,0])
relaxation(A, b, initial_guess, 25, 1e-8,1.2)